# **<font color='crimson'>Ranking</font>**

---


In [ ]:
###

In [ ]:
SELECT
    NTILE(2) OVER w AS tile,
    name,
    city,
    salary
FROM employees
WINDOW w AS (PARTITION BY city
    ORDER BY salary ASC)
ORDER BY city, salary, id;

In [ ]:
###

In [ ]:
WITH sorted_table AS (
SELECT
    DENSE_RANK() OVER w AS rank_index,
    id,
    name,
    department,
    salary
FROM employees
WINDOW w AS (
    PARTITION BY department
    ORDER BY salary DESC)
ORDER BY department, salary DESC, id
)

SELECT
    sorted_table.id,
    sorted_table.name,
    sorted_table.department,
    sorted_table.salary
FROM sorted_table
WHERE rank_index = 1;

# **<font color='crimson'>Offset</font>**

---

In [ ]:
###

In [ ]:
SELECT
    name,
    department,
    LAG(salary, 1) OVER w1 AS prev,
    salary,
    LEAD(salary, 1) OVER w2 AS next
FROM employees
WINDOW w1 AS (ORDER BY salary id),
    w2 AS (ORDER BY salary, id)
ORDER BY salary ASC, id ASC;

In [ ]:
###

In [ ]:
WITH temp_table AS (
SELECT
    name, city, salary,
    LAST_VALUE(salary) OVER w AS max_salary
FROM employees
WINDOW w AS (
    PARTITION BY city
    ORDER BY salary
    ROWS BETWEEN unbounded preceding AND unbounded following)
ORDER BY city ASC, salary ASC)

SELECT
    name, city, salary,
    ROUND(salary * 1.0 / max_salary * 100.0) AS percent
FROM temp_table
ORDER BY city ASC, salary ASC;

# **<font color='crimson'>Aggregation</font>**

---

In [ ]:
###

In [ ]:
SELECT
    name, city, salary,
    SUM(salary) OVER w AS fund,
    ROUND(salary * 100.0 / SUM(salary) OVER w) AS perc
FROM employees
WINDOW w AS (PARTITION BY city)
ORDER BY city ASC, salary ASC;

In [ ]:
###

In [ ]:
SELECT
    name, department, salary,
    COUNT(name) OVER w AS emp_cnt,
    ROUND(AVG(salary) OVER w) AS sal_avg,
    ROUND(salary * 100.0 / AVG(salary) OVER w - 100.0) AS diff
FROM employees
WINDOW w AS (PARTITION BY department)
ORDER BY department ASC, salary ASC, id ASC;

# **<font color='crimson'>Sliding Aggregates</font>**

---

In [ ]:
###

In [ ]:
SELECT
    year,
    month,
    income,
    AVG(income) over w AS roll_avg
FROM expenses
WINDOW w AS (
    ORDER BY year, month
    ROWS BETWEEN 1 preceding AND CURRENT ROW
)
ORDER BY year, month;

In [ ]:
###

In [ ]:
SELECT
    id,
    name,
    department,
    salary,
    SUM(salary) over w AS total
FROM employees
WINDOW w AS (
    PARTITION BY department
    ORDER BY department, salary, id
    ROWS BETWEEN unbounded preceding AND CURRENT ROW
)
ORDER BY department, salary, id;

# **<font color='crimson'>Statistics</font>**

---

In [ ]:
###

In [ ]:
SELECT
    wdate,
    wtemp,
    ROUND(cume_dist() OVER w, 2) AS cd,
    ROUND(percent_rank() OVER w, 2) AS pr
FROM weather
WHERE MONTH(wdate) = 3
WINDOW w AS (ORDER BY wtemp)
ORDER BY wtemp DESC
LIMIT 5;

In [ ]:
###

In [ ]:
SELECT
    wdate,
    wtemp,
    ROUND(cume_dist() OVER w, 2) AS perc
FROM weather
WHERE wdate BETWEEN '2020-03-01' AND '2020-03-31'
WINDOW w AS (ORDER BY wtemp)
ORDER BY wdate ASC
LIMIT 5;

In [ ]:
###

In [ ]:
WITH table_1 AS (
SELECT
    wdate,
    wtemp,
    MONTH(wdate) AS month_number
FROM weather
ORDER BY wdate ASC),

table_2 AS (
SELECT
    wdate,
    wtemp,
    month_number,
    ROUND(cume_dist() OVER w, 2) AS perc
FROM table_1
WINDOW w AS (
    PARTITION BY month_number
    ORDER BY wtemp)
ORDER BY wdate)

SELECT
    wdate,
    wtemp,
    perc
FROM table_2
WHERE DAY(wdate) = 7
ORDER BY wdate;

In [ ]:
###

In [ ]:
SELECT
    EXTRACT(MONTH FROM wdate) AS wmonth,
    ROUND(AVG(wtemp)::decimal, 2) AS t_avg,
    percentile_disc(0.50) WITHIN GROUP (ORDER BY wtemp) AS t_med,
    percentile_disc(0.90) WITHIN GROUP (ORDER BY wtemp) AS t_p90
FROM weather
GROUP BY wmonth
ORDER BY wmonth;